In [1]:
from openai import OpenAI
import os

In [2]:
client = OpenAI()

In [ ]:
# have to add at least $5 to OpenAI balance if encounter RateLimitError
response = client.chat.completions.create(
    model='gpt-5-mini',
    messages=[{"role":"user", "content":"is it too late to join the course?"}]

)
    

In [ ]:
response.choices[0].message.content

In [ ]:
!pip install minsearch

!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

In [5]:
import minsearch
import json

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

# a doc inside a course_dict only has 'text', 'section', 'question', but nothing like 'course', because in the json file it's already a block inside a specific course. To build our documents dictionary of doc objects, we get the 'course' info to add to each doc ob ject.
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
# build an index object so that when the user queries a specific term, the result is returned faster (compared to when searching through the whole document every time)
# there's more preprocessing time and memory space required to create an index object, but in turn we get much faster lookups especially when the number of documents grow

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)
index.fit(documents)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [9]:
q = 'the course has already started, can I still enroll?'

In [13]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
	query=q,
	filter_dict={'course': 'data-engineering-zoomcamp'},
	boost_dict=boost,
	num_results=3
)

results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202